# * VINSIGHT : Data Monitoring

## Parameter

In [11]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [12]:
''' Input parameter '''

op_dir = 'data'
op_monthly_file = f'vis_monthly_{str_curr_dt}'
op_daily_file = f'vis_daily_{str_curr_dt}'
op_ccaa_ccaatt_file = f'vis_ccaa_ccaatt_{str_curr_dt}'

v_year = 0
v_month_start = 0
v_month_end = 0
v_date = 20240101

# print(f"\nParameter input...\n\n   -> op_dir: {op_dir}\n   -> op_monthly_file: {op_monthly_file}\n   -> op_daily_file: {op_daily_file}")
print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_monthly_file: {op_monthly_file}')
print(f'   -> op_daily_file: {op_daily_file}')
print(f'   -> op_ccaa_ccaatt_file: {op_ccaa_ccaatt_file}')
print(f'\n   -> v_year: {v_year}')
print(f'   -> v_month_start: {v_month_start}')
print(f'   -> v_month_end: {v_month_end}')
print(f'   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_monthly_file: vis_monthly_20260126
   -> op_daily_file: vis_daily_20260126
   -> op_ccaa_ccaatt_file: vis_ccaa_ccaatt_20260126

   -> v_year: 0
   -> v_month_start: 0
   -> v_month_end: 0
   -> v_date: 20240101


## Aggregate : Timestamp

In [13]:
''' Execute : AGG_PERF_NEWCO '''


# Input parameter
v_start_date = 20260101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ PPN_TM, AREA_TYPE
        , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT
        , MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE CENTER IN ('Revenue', 'Sales', 'Profitability')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY PPN_TM, AREA_TYPE
    ORDER BY PPN_TM, AREA_TYPE
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    vis_timestamp_df = chk_src_df.copy()
    print(f'\n{vis_timestamp_df.to_string(max_cols=7)}') #max_rows=1000
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20260101

Data as of 2026-01-26, 13:46:36

Current Timestamp...

                PPN_TM AREA_TYPE  CNT_METRIC  ROW_CNT    MIN_DT    MAX_DT  LAST_ACTUAL
0  2026-01-25 10:24:37        HH         385   887040  20260101  20260124     20260123
1  2026-01-25 10:32:52        HH           4     9216  20260101  20260124     20260124
2  2026-01-26 04:25:54         C         275     6875  20260101  20260125     20260124
3  2026-01-26 04:25:54         G         432    86400  20260101  20260125     20260124
4  2026-01-26 04:25:54         P         553    13825  20260101  20260125     20260124
5  2026-01-26 04:25:54         Z          29     5075  20260101  20260125     20260116
6  2026-01-26 04:26:49         C           4      100  20260101  20260125     20260125
7  2026-01-26 04:26:49         G           4      800  20260101  20260125     20260125
8  2026-01-26 04:26:49         P           4      100  20260101  20260125     20260125
9  2026-01-26 10:14:55   

In [14]:
''' Execute : AGG_PERF_NEWCO_CCAA '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20260101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ AREA_TYPE
        , MAX(PPN_TM) PPN_TM
        , COUNT(1) ROW_CNT, MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
        , COUNT(DISTINCT METRIC_CD) CNT_METRIC
    FROM GEOSPCAPPO.AGG_PERF_NEWCO_CCAA 
    WHERE TM_KEY_DAY >= {v_start_date} 
    GROUP BY AREA_TYPE
    ORDER BY 1
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    vis_timestamp_df = chk_src_df.copy()
    print(f'\n{vis_timestamp_df.to_string(max_cols=7)}') #max_rows=1000
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2026-01-26, 13:50:05
   -> v_start_date: 20260101

Current Timestamp...

  AREA_TYPE              PPN_TM  ROW_CNT    MIN_DT    MAX_DT LAST_ACTUAL  CNT_METRIC
0      CCAA 2026-01-26 08:52:26  6490432  20260101  20260126        None         269


In [15]:
# ''' Execute : AGG_PERF_NEWCO_CCAATT '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20251101
# print(f'\nParameter input...\n')
# print(f'   -> Data as of {curr_datetime}')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ AREA_TYPE
#         , MAX(PPN_TM) PPN_TM
#         , COUNT(1) ROW_CNT, MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
#         , COUNT(DISTINCT METRIC_CD) CNT_METRIC
#     FROM GEOSPCAPPO.AGG_PERF_NEWCO_CCAATT 
#     WHERE TM_KEY_DAY >= {v_start_date} 
#     GROUP BY AREA_TYPE
#     ORDER BY 1
# """)


# try:
#     # Get : Timestamp
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     print(f'\nCurrent Timestamp...')
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

#     # Display
#     vis_timestamp_df = chk_src_df.copy()
#     print(f'\n{vis_timestamp_df.to_string(max_cols=7)}') #max_rows=1000
    
#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()